In [1]:
from IPython.display import display, HTML,clear_output
# css = """
# .output {
# flex-direction: row;
# flex-wrap: wrap;


# }
# """

# HTML('<style>{}</style>'.format(css))

In [2]:
import requests
import pandas as pd
import time
# from bs4 import BeautifulSoup

In [3]:
url ='https://www.nseindia.com/api/liveEquity-derivatives?index=nifty_bank_opt'
url_f ='https://www.nseindia.com/api/liveEquity-derivatives?index=nifty_bank_fut'

header = {
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/111.0.0.0 Safari/537.36",
    "Sec-Fetch-User": "?1", "Accept": "*/*", "Sec-Fetch-Site": "none", "Sec-Fetch-Mode": "navigate",
    "Accept-Encoding": "gzip, deflate, br", "Accept-Language": "en-US,en;q=0.9,hi;q=0.8"
    }
session = requests.session()
nse_live = session.get("http://nseindia.com", headers=header)


In [4]:
def generate_signals(df):
    signals = []


    for index, row in df.iterrows():
        # Extract relevant columns
        call_lp = row['ce_lp']
        put_lp = row['pe_lp']
        call_oi = row['ce_oi']
        put_oi = row['pe_oi']
        call_vol = row['ce_v']
        put_vol = row['pe_v']
        call_not = row['ce_not']  # Call noOfTrades
        put_not = row['pe_not']   # Put noOfTrades
        strike_price = row['strikePrice']

        # Signal determination logic using number of trades (ce_not and pe_not)
        if put_lp > call_lp and put_oi > call_oi and put_vol > call_vol:
            signal = 'Bearish'
        elif call_lp > put_lp and call_oi > put_oi and call_vol > put_vol:
            signal = 'Bullish'
        else:
            signal = 'Neutral'

        # Append the results for each strike price
        signals.append({
            'Strike Price': strike_price,
            'Call Price': call_lp,
            'Put Price': put_lp,
            'Call OI': call_oi,
            'Put OI': put_oi,
            'Call Volume': call_vol,
            'Put Volume': put_vol,
            'Call NoOfTrades': call_not,
            'Put NoOfTrades': put_not,
            'Signal': signal
        })

    # Append support and resistance levels to the DataFrame
    result_df = pd.DataFrame(signals)

    return result_df

In [5]:
def main():
    while True:
        clear_output(wait=True)
        request = session.get(url,headers=header).json()['data']
        df = pd.DataFrame(request).fillna(0)
        price = int(df['underlyingValue'][0])
        range_price = list(range(round(price/100)*100-800,round(price/100)*100+800,100))
        selected_columns = ['expiryDate', 'optionType', 'strikePrice', 'lastPrice', 'volume', 'totalTurnover', 
                            'value', 'premiumTurnOver', 'underlyingValue', 'openInterest', 'noOfTrades']
        selected_display = ['expiryDate_x','ce_lp', 'ce_v', 'ce_tt', 'ce_pt', 
                    'ce_U', 'ce_oi','ce_not', 'strikePrice' ,'pe_lp', 'pe_v', 'pe_tt', 'pe_pt', 
                    'pe_U', 'pe_oi', 'pe_not']
        df = df[selected_columns][(df['expiryDate']==df['expiryDate'].sort_values()[0])]
        df_pe = df[df['optionType']=='Put']
        df_pe.rename(columns={'lastPrice': 'pe_lp', 'volume': 'pe_v','totalTurnover': 'pe_tt', 'value': 'pe_value',
                                                                                'premiumTurnOver': 'pe_pt',
                            'underlyingValue': 'pe_U','openInterest':'pe_oi','noOfTrades':'pe_not'}, inplace=True)
        df_ce = df[df['optionType']=='Call']
        df_ce.rename(columns={'lastPrice': 'ce_lp', 'volume': 'ce_v','totalTurnover': 'ce_tt',
                                                                                'value': 'ce_value','premiumTurnOver': 'ce_pt',
                            'underlyingValue': 'ce_U','openInterest':'ce_oi','noOfTrades':'ce_not'}, inplace=True)
        df_merged = pd.merge(df_ce, df_pe, on='strikePrice', how='inner')
        df_merged = df_merged[selected_display][(df_merged['strikePrice'].isin(range_price))]
        df_sin = generate_signals(df_merged)
        support_level = df_merged.loc[df_merged['pe_oi'].idxmax(), 'strikePrice']
        print(f"Support Level: {support_level}")

        # Find the resistance level (highest Call OI)
        resistance_level = df_merged.loc[df_merged['ce_oi'].idxmax(), 'strikePrice']
        print(f"Resistance Level: {resistance_level}")

        # Optional: Display the entire DataFrame sorted by Put OI and Call OI for better insight
        print("Top 5 Support Levels (Highest Put OI):")
        print(df_merged[['strikePrice', 'pe_oi']].sort_values(by='pe_oi', ascending=False).head())

        print("Top 5 Resistance Levels (Highest Call OI):")
        print(df_merged[['strikePrice', 'ce_oi']].sort_values(by='ce_oi', ascending=False).head())
        display(df_merged.sort_values('strikePrice'),df_sin)
        time.sleep(2)

In [5]:
def main_run():
   try:
         clear_output(wait=True)
         main()
   except Exception as e:
         clear_output(wait=True)
         print(f'trying to retry to re-slove issue')
      #    raise e
         main_run()

main_run()

trying to retry to re-slove issue
